In [ ]:
from bottle import Bottle, route, run, template, get, post, debug, static_file, request, redirect, response
import time
import random
import string
import logging
import logging.handlers
import t_test
import sqlite3
import two_t_test

log = logging.getLogger('bottle')
log.setLevel('INFO')
h = logging.handlers.TimedRotatingFileHandler(
    'logs/nlog', when='midnight', backupCount=9999)
f = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
h.setFormatter(f)
log.addHandler(h)


secretKey = "vcxv&TTFS98v334sd123fsdhkldasdjghd6DFOUFYWE&FY"

from statics import messDict

@route('/static/<path:path>', name='static')
def static(path):
    return static_file(path, root='./static')


def checkAuth():
    from users import users
    loginName = request.get_cookie("user", secret=secretKey)
    password = request.get_cookie("password", secret=secretKey)
    randStr = request.get_cookie("randStr", secret=secretKey)
    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    users = {row[0]:{"password":row[1]} for row in c.execute("SELECT email, password FROM users")}
    conn.close()
    if (loginName in users) and users[loginName]["password"] == password:
        return loginName
    return redirect('/login')


@route('/login/')
@route('/login')
def login():
    return template("login")

@route('/login')
@route('/login/')
@route('/login', method='POST')
def login():
    loginName = request.forms.getunicode('inputEmail', default=False)
    password = request.forms.getunicode('inputPassword', default=False)
    randStr = ''.join(random.choice(
        string.ascii_uppercase + string.digits) for _ in range(18))
    
    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    users = {row[0]:{"password":row[1]} for row in c.execute("SELECT email, password FROM users")}
    conn.close()
    if (loginName in users) and users[loginName]["password"] == password:
        response.set_cookie("user", loginName, secret=secretKey)
        response.set_cookie("password", password, secret=secretKey)
        response.set_cookie("randStr", randStr, secret=secretKey)    
        redirect('/index')
        return True
    else:
        return template('login')
    return template('login')


@route('/')
@route('/index')
@route('/index/')
@route('/index/<message>')
def index(message=''):
    loginName = checkAuth()
    return template('index', message=messDict.get(message, ""), loginName=loginName)

@route('/register')
@route('/register/')
def register():
    return template("register")

@route('/register', method="POST")
@route('/register/', method="POST")
@route('/register', method = 'POST')
def register():
    name = request.forms.get('inputFirstName')
    surname = request.forms.get('inputLastName')
    email = request.forms.get('inputEmail')
    password = request.forms.get('inputPassword')
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    users = {row[0]:{"password":row[1]} for row in c.execute("SELECT email, password FROM users")}
    conn.close()
    if email in users:
        return template("register")
    else:
        conn = sqlite3.connect('users.db')
        c = conn.cursor()
        c.execute("INSERT INTO users (user_id, name, surname, email, password) VALUES (?,?,?,?,?)", (None, name, surname, email, password))
        conn.commit()
        conn.close()
        redirect('/login')
        return True
    return template("register")

@route('/one_sample_ttest')
@route('/one_sample_ttest/')
def one_sample_ttest():
    return template("one_sample_ttest")

@route('/one_sample_ttest', method = 'POST')
@route('/one_sample_ttest/', method = 'POST')
def one_sample_ttest():
    if request.POST.get('submit'):
        sample_mean = float(request.forms.get("inputSampleMean"))
        sample_std = float(request.forms.get("inputSampleStd"))
        test_type = request.forms.get("tail")
        hipothetical_mean = float(request.forms.get("inputHypothesizedMean"))
        N = float(request.forms.get("inputSampleSize"))
        alpha = float(request.forms.get("inputAlpha"))
        result = t_test.one_sample_t_test(sample_mean, sample_std, hipothetical_mean, N, alpha, test_type)
    return template("one_sample_ttest_results", result=result)

@route('/two_sample_ttest')
@route('/two_sample_ttest/')
def two_sample_ttest():
    return template("two_sample_ttest")

@route('/two_sample_ttest', method = 'POST')
@route('/two_sample_ttest/', method = 'POST')
def two_sample_ttest():
    if request.POST.get('submit'):
        first_sample_mean = float(request.forms.get("inputFirstSampleMean"))
        first_sample_std = float(request.forms.get("inputFirstSampleStd"))
        second_sample_mean = float(request.forms.get("inputSecondSampleMean"))
        second_sample_std = float(request.forms.get("inputSecondSampleStd"))
        test_type = request.forms.get("tail")
        first_N = float(request.forms.get("inputFirstSampleSize"))
        second_N = float(request.forms.get("inputSecondSampleSize"))
        alpha = float(request.forms.get("inputAlpha"))
        result = two_t_test.two_sample_t_test(first_sample_mean, first_sample_std, second_sample_mean, second_sample_std, first_N, second_N, alpha, test_type)
    return template("two_sample_ttest_results", result=result)

@route('/paired_ttest')
@route('/paired_ttest/')
def paired_ttest():
    return template("paired_ttest")

@route('/paired_ttest', method = "POST")
@route('/paired_ttest/', method = "POST")
def paired_ttest():
    if request.POST.get("submit"):
        diff_mean = float(request.forms.get("inputSampleMean"))
        diff_std = float(request.forms.get("inputSampleStd"))
        test_type = request.forms.get("tail")
        hipothetical_mean = float(request.forms.get("inputHypothesizedMean"))
        N = float(request.forms.get("inputSampleSize"))
        alpha = float(request.forms.get("inputAlpha"))
        result = t_test.paired_ttest(diff_mean, diff_std, hipothetical_mean, N, alpha, test_type)
    return template("paired_ttest_results", result = result)

@route('/static/<path:path>', name='static')
def static(path):
    return static_file(path, root='./static')

# Debug = True - changes in template are visible immidietly
run(host='localhost', port=8022, debug=True)

Bottle v0.12.19 server starting up (using WSGIRefServer())...
Listening on http://localhost:8022/
Hit Ctrl-C to quit.

127.0.0.1 - - [18/Jan/2022 22:48:55] "GET / HTTP/1.1" 200 5519
127.0.0.1 - - [18/Jan/2022 22:49:00] "GET /login HTTP/1.1" 200 3479
127.0.0.1 - - [18/Jan/2022 22:49:08] "POST /login HTTP/1.1" 303 0
127.0.0.1 - - [18/Jan/2022 22:49:08] "GET /index HTTP/1.1" 200 5519
127.0.0.1 - - [18/Jan/2022 22:49:09] "GET /one_sample_ttest HTTP/1.1" 200 7891
127.0.0.1 - - [18/Jan/2022 22:49:11] "GET /two_sample_ttest HTTP/1.1" 200 8742
127.0.0.1 - - [18/Jan/2022 22:49:12] "GET /paired_ttest HTTP/1.1" 200 7905
127.0.0.1 - - [18/Jan/2022 22:49:32] "POST /one_sample_ttest HTTP/1.1" 200 6450
127.0.0.1 - - [18/Jan/2022 22:49:32] "GET /static/ttestimg.img HTTP/1.1" 404 735
127.0.0.1 - - [18/Jan/2022 22:50:04] "GET /login HTTP/1.1" 200 3479
